In [275]:
!pip install pandas
!pip install numpy
!pip install matplotlib
!pip install seaborn

In [276]:
!pip install scikit-learn
!pip install tensorflow
!pip install nltk
!pip install beautifulsoup4

In [277]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [278]:
df = pd.read_csv("..\\data\\raw\\1.6M_twitter.csv", encoding='latin1', header=None, names=['target', 'ids', 'date', 'flag', 'user', 'text'])
df.head()

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


### Cleaning

In [279]:
df['target'] = df['target'].map({0: 0, 4: 1})
df['target'].value_counts()

target
0    800000
1    800000
Name: count, dtype: int64

In [280]:
# find users with tweets more than 50
df.groupby('user').size().sort_values(ascending=False).head(50)

user
lost_dog           549
webwoke            345
tweetpet           310
SallytheShizzle    281
VioletsCRUK        279
mcraddictal        276
tsarnick           248
what_bugs_u        246
Karen230683        238
DarkPiano          236
SongoftheOss       227
Jayme1988          225
keza34             219
ramdomthoughts     216
shanajaca          213
wowlew             212
nuttychris         211
TraceyHewins       211
thisgoeshere       207
Spidersamm         205
StDAY              202
felicityfuller     195
Dogbook            192
_magic8ball        189
Djalfy             182
Dutchrudder        182
torilovesbradie    182
twebbstack         180
Quimo              180
Broooooke_         179
enamoredsoul       179
MTVnHollyWEST23    178
JessMcFlyxxx       178
MiDesfileNegro     177
KevinEdwardsJr     171
linnetwoods        171
insearchofnkotb    170
Scyranth           166
karinb_za          166
JBnVFCLover786     163
cookiemonster82    160
maynaseric         159
shellrawlins       159
hollya

In [281]:
df.drop(['ids', 'date', 'flag', 'user'], axis=1, inplace=True)

In [282]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 2 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   target  1600000 non-null  int64 
 1   text    1600000 non-null  object
dtypes: int64(1), object(1)
memory usage: 24.4+ MB


In [283]:
df['text'] = df['text'].str.replace(r'@\w+', '', regex=True)  # Remove @mentions
df['text'] = df['text'].str.replace(r'https?://\S+', '', regex=True)  # Remove URLs
df.head()

,target,text
0,0,"- Awww, that's a bummer. You shoulda got Da..."
1,0,is upset that he can't update his Facebook by ...
2,0,I dived many times for the ball. Managed to s...
3,0,my whole body feels itchy and like its on fire
4,0,"no, it's not behaving at all. i'm mad. why am..."


In [284]:
df['text'] = df['text'].str.replace('&quot;', '') # Remove &quot;
df.head()

,target,text
0,0,"- Awww, that's a bummer. You shoulda got Da..."
1,0,is upset that he can't update his Facebook by ...
2,0,I dived many times for the ball. Managed to s...
3,0,my whole body feels itchy and like its on fire
4,0,"no, it's not behaving at all. i'm mad. why am..."


In [285]:
df[df['text'].str.contains('&quot;')] # validate

,target,text


In [286]:
df['text'] = df['text'].str.replace('&lt;', ' ')  #remove &lt; (less than)
df[df['text'].str.contains('&lt;')]

,target,text


In [287]:
df['text'] = df['text'].str.replace('&gt;', ' ')  #remove &gt; (greater than)
df[df['text'].str.contains('&gt;')]

,target,text


In [288]:
df['text'] = df['text'].str.replace('--+', '', regex=True) # Remove multiple dashes
df[df['text'].str.contains('--+')]

,target,text


In [289]:
df['text'] = df['text'] = df['text'].str.replace('&amp;', 'and')  #replace &amp; with and
df[df['text'].str.contains('&amp;')] # validate

,target,text


In [290]:
df['text'].str.strip()  # Remove leading/trailing whitespaces

0          - Awww, that's a bummer.  You shoulda got Davi...
1          is upset that he can't update his Facebook by ...
2          I dived many times for the ball. Managed to sa...
3             my whole body feels itchy and like its on fire
4          no, it's not behaving at all. i'm mad. why am ...
                                 ...                        
1599995    Just woke up. Having no school is the best fee...
1599996    TheWDB.com - Very cool to hear old Walt interv...
1599997    Are you ready for your MoJo Makeover? Ask me f...
1599998    Happy 38th Birthday to my boo of alll time!!! ...
1599999                                happy #charitytuesday
Name: text, Length: 1600000, dtype: object

In [291]:
df.head()

,target,text
0,0,"- Awww, that's a bummer. You shoulda got Da..."
1,0,is upset that he can't update his Facebook by ...
2,0,I dived many times for the ball. Managed to s...
3,0,my whole body feels itchy and like its on fire
4,0,"no, it's not behaving at all. i'm mad. why am..."


In [292]:
df.drop_duplicates(inplace=True)

In [293]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1558626 entries, 0 to 1599999
Data columns (total 2 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   target  1558626 non-null  int64 
 1   text    1558626 non-null  object
dtypes: int64(1), object(1)
memory usage: 35.7+ MB


### Split

In [294]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)
x_train, y_train = np.array(train_data['text']), np.array(train_data['target'])
x_test, y_test = np.array(test_data['text']), np.array(test_data['target'])

#Print the sizes of the train, dev, and test sets
print("Train data shape:", x_train.shape , y_train.shape)
print("Test data shape:", x_test.shape, y_test.shape)

Train data shape: (1246900,) (1246900,)
Test data shape: (311726,) (311726,)


In [295]:
print(x_train[0], y_train[0])
print(x_test[0], y_test[0])

 So they still cant find them?? Thats tragic...  0
tia and/or tamera from sister, sister is on a tbn commercial. I love it when I see celebrities praising God. I love it.  1


### Model

In [296]:
train_doc_length = 0
for word in x_train:
  tokens = str(word).split()
  train_doc_length += len(tokens)

print('\nTraining data average document length =', (train_doc_length / len(x_train)))


Training data average document length = 12.857509824364424


In [297]:
import tensorflow as tf

VOCAB_SIZE = 3000000
SEQ_MAX_LENGTH = 20
vectorizer = tf.keras.layers.TextVectorization(max_tokens=VOCAB_SIZE, 
                                               output_mode='int', 
                                               ngrams=1, name='vector_text',
                                               output_sequence_length=SEQ_MAX_LENGTH)

with tf.device('/CPU:0'):
  vectorizer.adapt(x_train)

### Visualizations

In [298]:
from sklearn.metrics import classification_report, accuracy_score, f1_score, precision_score, recall_score, roc_curve, roc_auc_score, auc
from matplotlib import pyplot as plt
from matplotlib.ticker import FormatStrFormatter
from seaborn import heatmap
from pandas import DataFrame
import IPython.display as ipd


def classification_data_nn(estimator, 
                          x_train, y_train,
                          x_test, y_test, 
                          epochs=1, 
                          num_layers=1,
                          emb_size=64,
                          h_size=64,
                          batch_size=64,
                          splits=5):
  
  train_accuracies, test_accuracies, train_precisions, test_precisions, train_recall, test_recall, train_f1, test_f1 = [], [], [], [], [], [], [], []
  
  split_size = int(len(x_train) / splits)
  x_splits = np.split(x_train, splits)
  y_splits = np.split(y_train, splits)
  test_cm = None
  
  for i in range(0, len(x_splits)):
    if i == 0:
      curr_x = x_splits[0]
      curr_y = y_splits[0]
    else:
      curr_x = np.concatenate((curr_x, x_splits[i]), axis=0)
      curr_y = np.concatenate((curr_y, y_splits[i]), axis=0)
    
    new_estimator = estimator(num_layers=num_layers, emb_size=emb_size, h_size=h_size)
    model = new_estimator.get_model()
    est_his = model.fit(curr_x, curr_y, epochs=epochs, batch_size=batch_size)
    
    train_pred = model.predict(curr_x)
    test_pred = model.predict(x_test)
    
    train_pred = np.round(train_pred)
    test_pred = np.round(test_pred)

    train_accuracies.append(accuracy_score(curr_y, train_pred))
    test_accuracies.append(accuracy_score(y_test, test_pred))
    
    train_precisions.append(precision_score(curr_y, train_pred))
    test_precisions.append(precision_score(y_test, test_pred))
    
    train_recall.append(recall_score(curr_y, train_pred))
    test_recall.append(recall_score(y_test, test_pred))
    
    train_f1.append(f1_score(curr_y, train_pred))
    test_f1.append(f1_score(y_test, test_pred))


  return {'estimator': new_estimator.__name__, 
          'splits': splits,
          'split_size': split_size, 
          'test_predictions': test_pred,
          'test_predictions_cont': test_pred,
          'train_accuracy': train_accuracies, 
          'test_accuracy': test_accuracies, 
          'train_precision': train_precisions, 
          'test_precision': test_precisions, 
          'train_recall': train_recall, 
          'test_recall': test_recall, 
          'train_f1': train_f1, 
          'test_f1': test_f1,
          }

def classification_data(estimator, 
                          x_train, y_train,
                          x_test, y_test,
                          splits = 5):
  train_accuracies, test_accuracies, train_precisions, test_precisions, train_recall, test_recall, train_f1, test_f1 = [], [], [], [], [], [], [], []
  
  # Split the training data into n splits
  split_size = int(len(x_train) / splits)
  x_splits = np.split(x_train, splits)
  y_splits = np.split(y_train, splits)
  
  # Train the model on each split and evaluate on the test set
  for i in range(0, len(x_splits)):
    if i == 0:
      curr_x = x_splits[0]
      curr_y = y_splits[0]
    else:
      curr_x = np.concatenate((curr_x, x_splits[i]), axis=0)
      curr_y = np.concatenate((curr_y, y_splits[i]), axis=0)
    
    # Train the model and get train/test predictions
    estimator.fit(curr_x, curr_y)
    train_pred = estimator.predict(curr_x)
    test_pred = estimator.predict(x_test)
    
    # Calculate and save the necessary metrics for this train/test split
    train_accuracies.append(accuracy_score(curr_y, train_pred))
    test_accuracies.append(accuracy_score(y_test, test_pred))
    
    train_precisions.append(precision_score(curr_y, train_pred))
    test_precisions.append(precision_score(y_test, test_pred))
    
    train_recall.append(recall_score(curr_y, train_pred))
    test_recall.append(recall_score(y_test, test_pred))
    
    train_f1.append(f1_score(curr_y, train_pred))
    test_f1.append(f1_score(y_test, test_pred))

  
  # Results required for all the future plots/tables
  return {'estimator': estimator.__class__.__name__, 
          'split_size': split_size, 
          'splits': splits,
          'test_predictions': test_pred,
          'train_accuracy': train_accuracies, 
          'test_accuracy': test_accuracies, 
          'train_precision': train_precisions, 
          'test_precision': test_precisions, 
          'train_recall': train_recall, 
          'test_recall': test_recall, 
          'train_f1': train_f1, 
          'test_f1': test_f1}
  
def classification_plots(classification_data, full_scale=False):
  split_size = classification_data['split_size']
  splits = classification_data['splits']
  
  figure, axis = plt.subplots(2, 2, figsize=(6, 6), dpi=100, gridspec_kw={'width_ratios': [1, 1], 'height_ratios': [1, 1]})
  figure.suptitle("Learning Curve for {estimator}".format(estimator=classification_data['estimator']), fontsize=16)
  labels = ['Accuracy', 'Precision', 'Recall', 'F1']
  
  for i in range(0, 2):
    for j in range(0, 2):
      axis[i, j].set_title(labels[i * 2 + j])
      axis[i, j].yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
      if full_scale:
        axis[i, j].axis(ymin=0, ymax=1.02)
      axis[i, j].plot(list(range(split_size, splits*split_size + split_size, split_size)), classification_data['train_' + labels[i * 2 + j].lower().replace(' ', '_')], '-', color="#2c8dc9", label="Training")
      axis[i, j].plot(list(range(split_size, splits*split_size + split_size, split_size)), classification_data['test_' + labels[i * 2 + j].lower().replace(' ', '_')], '-', color="#FFAD00", label="Testing")
      axis[i, j].grid(alpha = 0.3)  
  
  handles, labels = axis[1, 1].get_legend_handles_labels()
  figure.legend(handles, labels, loc='lower center', bbox_to_anchor=(0.5, -0.05), ncol=2, fancybox=True, shadow=True)
  figure.tight_layout()
 
  return figure


def classification_table(classification_data):
  split_size = classification_data['split_size']
  df = DataFrame(data={'Train Accuracy': np.round(classification_data['train_accuracy'], 2), 
                         'Test Accuracy': np.round(classification_data['test_accuracy'], 2), 
                         'Precision Train' : np.round(classification_data['train_precision'], 2), 
                         'Precision Test' : np.round(classification_data['test_precision'], 2), 
                         'Recall Train' : np.round(classification_data['train_recall'], 2), 
                         'Recall Test' : np.round(classification_data['test_recall'], 2), 
                         'F1 Train' : np.round(classification_data['train_f1'], 2), 
                         'F1 Test' : np.round(classification_data['test_f1'], 2)}, 
                   index=list(range(split_size, len(x_train) + split_size, split_size)))
  return df

def classification_plots_compare(classification_data_x, classification_data_y, full_scale=False):
  """
  Plots the learning curves for the train/test accuracies, precisions, recalls
  and F1 scores for each split in one figure for both classifiers.
  
  Arguments:
    classification_data_x: The dictionary containing the train/test data for the first classifier.
    classification_data_y: The dictionary containing the train/test data for the second classifier.
    full_scale: Whether or not to plot the full scale of the y-axis.
  Returns: 
    A figure containing the learning curves for the train/test accuracies, precisions, recalls and F1 scores 
    for both classifiers.
  """
  
  split_size = classification_data_x['split_size']
  splits = classification_data_x['splits']
  
  figure, axis = plt.subplots(2, 2, figsize=(6, 6), dpi=100, gridspec_kw={'width_ratios': [1, 1], 'height_ratios': [1, 1]})
  figure.suptitle("Learning Curve Comparison for {estimator} against {estimator_2} ".format(estimator=classification_data_x['estimator'], estimator_2=classification_data_y['estimator']), fontsize=12)
  labels = ['Accuracy', 'Precision', 'Recall', 'F1']
  
  for i in range(0, 2):
    for j in range(0, 2):
      axis[i, j].set_title(labels[i * 2 + j])
      axis[i, j].yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
      if full_scale:
        axis[i, j].axis(ymin=0, ymax=1.02)
      axis[i, j].plot(list(range(split_size, splits*split_size + split_size, split_size)), classification_data_y['train_' + labels[i * 2 + j].lower().replace(' ', '_')], '-', color="#AD49C2", label="Training {estimator}".format(estimator=classification_data_y['estimator']))
      axis[i, j].plot(list(range(split_size, splits*split_size + split_size, split_size)), classification_data_y['test_' + labels[i * 2 + j].lower().replace(' ', '_')], '-', color="#7CC249", label="Testing {estimator}".format(estimator=classification_data_y['estimator']))
      axis[i, j].plot(list(range(split_size, splits*split_size + split_size, split_size)), classification_data_x['train_' + labels[i * 2 + j].lower().replace(' ', '_')], '-', color="#2c8dc9", label="Training {estimator}".format(estimator=classification_data_x['estimator']))
      axis[i, j].plot(list(range(split_size, splits*split_size + split_size, split_size)), classification_data_x['test_' + labels[i * 2 + j].lower().replace(' ', '_')], '-', color="#FFAD00", label="Testing {estimator}".format(estimator=classification_data_x['estimator']))
      axis[i, j].grid(alpha = 0.3) 
    
  handles, labels = axis[1, 1].get_legend_handles_labels()
  figure.legend(handles, labels, loc='lower center', bbox_to_anchor=(0.5, -0.1), ncol=2, fancybox=True, shadow=True)
  figure.tight_layout()
  return figure


def roc_curve_plot(y_pred_cont, name):
  fpr, tpr, _ = roc_curve(y_test, y_pred_cont)
  roc_auc = auc(fpr, tpr)

  figure, axis = plt.subplots(1, 1, figsize=(6, 6), dpi=100)
  axis.plot(fpr, tpr, color='#2c8dc9', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
  axis.plot([0, 1], [0, 1], color='#FFAD00', lw=2, linestyle='--', label='No Skill')
  axis.set_xlim([0.0, 1.0])
  axis.set_ylim([0.0, 1.05])
  axis.set_xlabel('False Positive Rate')
  axis.set_ylabel('True Positive Rate')
  axis.set_title('Receiver operating characteristic for {estimator}'.format(estimator=name))
  axis.legend(loc="lower right")
  axis.grid(alpha = 0.3)

  return figure

def loss_plot(loss, val_loss, name):
  figure, axis = plt.subplots(1, 1, figsize=(6, 6), dpi=100)
  epochs = range(1, len(loss)+1)
  axis.plot(epochs, loss, color='#2c8dc9', lw=2, label='Loss')
  axis.plot(epochs, val_loss, color='#FFAD00', lw=2, label='Validation Loss')
  axis.set_xlabel('Epoch')
  axis.set_ylabel('Loss')
  axis.set_title('Loss over Epochs for {estimator}'.format(estimator=name))
  axis.legend(loc="upper right")
  axis.grid(alpha = 0.3)

  return figure

### Model

In [299]:
class bigru_rnn():
    
    def __init__(self, num_layers=1, emb_size=64, h_size=64):
        self.num_layers = num_layers
        self.emb_size = emb_size
        self.h_size = h_size
        
        inputs = tf.keras.layers.Input(shape=(1,), dtype=tf.string, name='txt_input')
        x = vectorizer(inputs)
        x = tf.keras.layers.Embedding(input_dim=len(vectorizer.get_vocabulary()),
                                        output_dim=self.emb_size, name='word_embeddings',
                                        mask_zero=True)(x)
        
        for n in range(self.num_layers):
            if n != self.num_layers - 1:
                x = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(units=self.h_size, 
                                    name=f'bigru_cell_{n}', 
                                    return_sequences=True,
                                    dropout=0.2))(x)
            else:
                x = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(units=self.h_size, 
                                                name=f'bigru_cell_{n}',
                                                dropout=0.2))(x)
        x = tf.keras.layers.Dropout(rate=0.5)(x)
         
        o = tf.keras.layers.Dense(units=1, activation='sigmoid', name='lr')(x)
        
        self.model = tf.keras.models.Model(inputs=inputs, outputs=o, name='biGRU_RNN')
        self.model.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.BinaryCrossentropy())

    def __name__(self):
        return 'biGRU RNN'
    
    def fit(self, x_train, y_train, epochs=1, batch_size=64, validation_split=0):
        self.model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=validation_split)
    
    def get_model(self):
        return self.model
      
loss_rnn = bigru_rnn(num_layers=1, emb_size=64, h_size=64)
model = loss_rnn.get_model()
print(model.summary())

Model: "biGRU_RNN"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ txt_input           │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ vector_text         │ (None, 20)        │          0 │ txt_input[0][0]   │
│ (TextVectorization) │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ word_embeddings     │ (None, 20, 64)    │ 23,700,736 │ vector_text[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal_5         │ (None, 20)        │          0 │ vector_text[0][0] │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional_5     │ (None, 128)       │     49,920 │ word_embeddings[… │
│ (Bidirectional)     │                   │            │ not_equal_5[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_5 (Dropout) │ (None, 128)       │          0 │ bidirectional_5[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lr (Dense)          │ (None, 1)         │        129 │ dropout_5[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 23,750,785 (90.60 MB)

 Trainable params: 23,750,785 (90.60 MB)

 Non-trainable params: 0 (0.00 B)

None


In [ ]:
rnn_hist = model.fit(x_train, y_train, epochs=6, batch_size=128, validation_split=0.2)
rnn_pred = model.predict(x_test)

In [ ]:
rnn_ls_plt = loss_plot(rnn_hist.history['loss'], rnn_hist.history['val_loss'], model.name)
rnn_auc_plt = roc_curve_plot(rnn_pred, model.name)

# from tensorflow.keras.utils import plot_model
# from IPython.display import Image 
# plot_model(model, to_file='model.png', show_shapes=True)
# Image('model.png')